In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import datetime
import time
from importlib import reload
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import pandas
import glob
import os
import scipy.signal
from scipy import optimize 
import resonator_model
import read_stream_data_gcp_save

In [3]:
from __future__ import unicode_literals

figsize(8,6)

matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.unicode'] = True
    
rcParams['xtick.labelsize'] = 20
rcParams['xtick.major.size'] = 10
rcParams['xtick.major.width'] = 1
rcParams['xtick.major.pad'] = 10
rcParams['xtick.minor.size'] = 5
rcParams['xtick.minor.width'] = 1

rcParams['ytick.labelsize'] = 20
rcParams['ytick.major.size'] = 10
rcParams['ytick.major.width'] = 1
rcParams['ytick.major.pad'] = 10
rcParams['ytick.minor.size'] = 5
rcParams['ytick.minor.width'] = 1

rcParams['legend.fontsize'] = 12
rcParams['legend.borderaxespad'] = 2

rcParams['axes.labelsize'] = 24
rcParams['axes.titlesize'] = 24

C:\Users\Heather\Anaconda3\lib\site-packages\matplotlib\__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


In [4]:
#finds the nearest value in an array
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [5]:
pwd

'C:\\Users\\Heather\\Documents\\Git\\so_readout_and_detectors\\SMuRF\\201909_mmbv2pp'

In [7]:
#all the tuning files
tune_files = glob.glob('tune/*tune.npy')

In [8]:
#all the timestream files 
files = glob.glob('1568315085/outputs/*part*')

In [10]:
files

['1568315085/outputs\\1568315467.dat.part_00000',
 '1568315085/outputs\\1568315594.dat.part_00000',
 '1568315085/outputs\\1568315731.dat.part_00000',
 '1568315085/outputs\\1568315873.dat.part_00000',
 '1568315085/outputs\\1568316015.dat.part_00000',
 '1568315085/outputs\\1568316142.dat.part_00000',
 '1568315085/outputs\\1568316270.dat.part_00000',
 '1568315085/outputs\\1568316412.dat.part_00000',
 '1568315085/outputs\\1568316541.dat.part_00000',
 '1568315085/outputs\\1568316684.dat.part_00000',
 '1568315085/outputs\\1568316826.dat.part_00000',
 '1568315085/outputs\\1568316969.dat.part_00000',
 '1568315085/outputs\\1568317111.dat.part_00000',
 '1568315085/outputs\\1568317254.dat.part_00000',
 '1568315085/outputs\\1568317395.dat.part_00000',
 '1568315085/outputs\\1568317526.dat.part_00000',
 '1568315085/outputs\\1568317668.dat.part_00000',
 '1568315085/outputs\\1568317799.dat.part_00000',
 '1568315085/outputs\\1568317941.dat.part_00000',
 '1568315085/outputs\\1568318071.dat.part_00000',


In [12]:
def get_all_tune_timestamps(tune_files):
    #pass a list of tunefiles and 
    #get a dict of the timestamps associated with the tune files
    timestamps = []
    for tune_file in tune_files:
        tune_file_only = os.path.basename(tune_file)
        tune_ts = int(tune_file_only.split('_')[0])
        timestamps.append(tune_ts)
    tune_file_dict = dict(zip(timestamps, tune_files))
    return(tune_file_dict)

In [95]:
tune_file_dict = get_all_tune_timestamps(tune_files)

In [96]:
def get_all_noise_timestamps(noise_files):
    #pass a list of noise files and 
    #get a dict of the timestamps associated with them
    timestamps = []
    for noise_file in noise_files:
        noise_file_only = os.path.basename(noise_file)    
        noise_ts = int(noise_file_only.split('.')[0])
        timestamps.append(noise_ts)
    noise_file_dict = dict(zip(timestamps, noise_files))
    return(noise_file_dict)

In [97]:
noise_file_dict = get_all_noise_timestamps(files)

In [98]:
#create dataframe for associating the files
d_file = {'tune_file':[],
    'noise_file':[]}

d_file = pd.DataFrame(d_file)
d_file

,tune_file,noise_file


In [99]:
def find_nearest_tune_file(noise_file_dict, tune_file_dict, d_file):
    noise_ts_arr = np.array(list(noise_file_dict.keys()))
    tune_ts_arr = np.array(list(tune_file_dict.keys()))
    
    for noise_ts in noise_ts_arr:
        try:
            closest_tune_ts = tune_ts_arr[tune_ts_arr < noise_ts].max()
            tune_file = tune_file_dict[closest_tune_ts]
            noise_file = noise_file_dict[noise_ts]
            d_file = d_file.append({'tune_file': tune_file, 'noise_file': noise_file}, ignore_index=True)
        except:
            pass
    return d_file

In [100]:
d_file = find_nearest_tune_file(noise_file_dict, tune_file_dict, d_file)

In [13]:
d_file[30:50]

NameError: name 'd_file' is not defined